In [1]:
import urllib.request
import json
import math

url = 'https://data.cityofnewyork.us/resource/xyye-rtrs.json'

# Fetch the data from the URL
with urllib.request.urlopen(url) as url:
    data = json.loads(url.read().decode())

In [2]:
# Build dictionary
coordinates_to_neighborhood = {}
for neighborhood in data:
    if neighborhood['borough'] == "Manhattan":
        lat = neighborhood['the_geom']['coordinates'][1]
        lon = neighborhood['the_geom']['coordinates'][0]
        coordinates_to_neighborhood[tuple([lat, lon])] = neighborhood['name']

In [3]:
import pandas as pd
df_data = pd.DataFrame(list(coordinates_to_neighborhood.items()), columns=['coordinates', 'location'])

In [4]:
location_list = list(df_data['location'])
corodinate_list = list(df_data['coordinates'])

In [5]:
import folium
import branca.colormap as cm

# Define the list of neighborhoods and their corresponding numbers
neighborhoods = location_list
neighborhood_numbers = list(range(39)) 


# Define the latitude and longitude of Manhattan's center
manhattan_lat, manhattan_lon = 40.7831, -73.9712

# Create a map of Manhattan with CartoDB's Positron tiles
#Without color
m = folium.Map(location=[manhattan_lat, manhattan_lon], zoom_start=13, tiles='CartoDB Positron')
#with color
#m = folium.Map(location=[manhattan_lat, manhattan_lon], zoom_start=13)

# Create a colormap based on the range of neighborhood_numbers
color_scale = cm.LinearColormap(
    colors=["red", "yellow", "green", "blue"],
    vmin=min(neighborhood_numbers),
    vmax=max(neighborhood_numbers),
    caption="Neighborhood Numbers"
)

# Add neighborhood markers to the map
for neighborhood, number, (lat, lon) in zip(neighborhoods, neighborhood_numbers, corodinate_list):
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        color=color_scale(number),
        fill=True,
        fill_opacity=0.7
    ).add_to(m)
    
    folium.map.Marker(
        [lat, lon],
        icon=folium.DivIcon(icon_size=(150,36), icon_anchor=(0,0),
        html='<div style="font-size: 8.5pt; color: black;">{0}: {1}</div>'.format(neighborhood, number))
    ).add_to(m)

# Add the colormap legend to the map
color_scale.add_to(m)

# Save the map as an HTML file
m.save("manhattan_map_colored.html")